In [7]:
import paho.mqtt.client as mqtt
import numpy as np
import io
from IPython.display import clear_output
from warnings import warn
import time
global all_T_mc, all_R_mc
all_T_mc, all_R_mc = None, None

from scipy.spatial.transform import Rotation as R

In [2]:
def recv_markers(client, userdata, msg):
    global all_T_mc, all_R_mc
    
    data = np.load(io.BytesIO(msg.payload))    
    all_T_mc, all_R_mc = data['tvecs'], data['rvecs']

client = mqtt.Client()
client.on_message = recv_markers
client.connect('172.31.1.150', 1883)
client.subscribe('putzini/markers')
client.loop_start()

In [3]:
# DEFINE TRANSFORMS
# _ab means: "a as seen from b", or equivalently "transforms from a system from b system"

all_T_mr = np.ones((50, 3))*np.nan
all_R_mr = np.ones((50, 3, 3))*np.nan

all_T_mr[0,...] = np.array([[0, 0, -235]])
all_T_mr[10,...] = np.array([[0, 100, -235]])
all_T_mr[11,...] = np.array([[0, 100, -235]])
all_T_mr[12,...] = np.array([[0, 100, -235]])

all_R_mr[0,...] = np.eye(3)
all_R_mr[10,...] = np.eye(3)
all_R_mr[11,...] = np.eye(3)
all_R_mr[12,...] = np.eye(3)

T_cp = np.zeros(3)
R_cp = np.eye(3)

In [4]:
# Room coords vs Putzini coords
# coordinates with _a means "with respect to a"

X_m = lambda X_p: trans(R_mr, T_mr, False,
                       trans(R_mc, T_mc, True,
                            trans(R_cp, T_cp, True, X_p)))

In [5]:
def trans(R=None, T=None, inv=False, X=None):
    R = np.eye(3) if R is None else R
    T = np.zeros(3) if T is None else T
    X = np.zeros(3) if X is None else X
    if inv:
        return np.dot(np.linalg.inv(R), X-T)
    else:
        return np.dot(R, X)+T

In [6]:
def trans_all(X_p=None):
    global all_T_mc, all_R_mc
    
    X_p = np.zeros(3) if X_p is None else X_p
    valid = np.any(np.isnan(all_T_mc), axis=1) | np.any(np.isnan(all_T_mr), axis=1)
    valid = np.nonzero(1-valid)[0]
    
    all_X_c = np.ones_like(all_T_mr) * np.nan
    all_X_m = np.ones_like(all_T_mr) * np.nan
    all_X_r = np.ones_like(all_T_mr) * np.nan
    all_R_pm = np.ones_like(all_R_mr) * np.nan
    all_R_pr = np.ones_like(all_R_mr) * np.nan
    
    for ii in valid:
        T_mc, R_mc = all_T_mc[ii,...], all_R_mc[ii,...]
        T_mr, R_mr = all_T_mr[ii,...], all_R_mr[ii,...]
        
        all_X_c[ii,...] = trans(R_cp, T_cp, True, X_p)
        all_X_m[ii,...] = trans(R_mc, T_mc, True, all_X_c[ii,...])
        all_X_r[ii,...] = trans(R_mr, T_mr, False, all_X_m[ii,...])
        all_R_pm[ii,...] = np.dot(np.linalg.inv(R_mc), np.linalg.inv(R_cp))
        all_R_pr[ii,...] = np.dot(R_mr, all_R_pm[ii,...])
        
    return valid, all_X_c, all_X_m, all_X_r, all_R_pm, all_R_pr

In [ ]:
euler_convention = 'ZYX'

while True:

    clear_output(wait=True)
    
    pos = trans_all()
    if len(pos[0]) == 0:
        warn('Not seeing any marker!')
        continue
        
    valid_pos = [pos[0]] + [p[pos[0],...] for p in pos[1:]]
    pos_vs_room = valid_pos[3]/100
    pos_vs_markers = valid_pos[2]/100
    euler_vs_room = np.stack([R.from_dcm(r).as_euler(euler_convention, degrees=True) 
                              for r in valid_pos[5]])
    euler_vs_markers = np.stack([R.from_dcm(r).as_euler(euler_convention, degrees=True) 
                              for r in valid_pos[4]])

    print('Valid markers:', pos[0])
    print('Positions vs Room:\n', pos_vs_room)
    print('Euler angles vs Room:\n', euler_vs_room)

    print('Positions vs Markers:\n', pos_vs_markers)
    print('Euler angles vs Markers:\n', euler_vs_room)

    time.sleep(0.2)

Valid markers: [10]
Positions vs Room:
 [[-0.14576737 -4.75006776  2.34623242]]
Euler angles vs Room:
 [[-10.03851316 -10.08802245 173.11583432]]
Positions vs Markers:
 [[-0.14576737 -5.75006776  4.69623242]]
Euler angles vs Markers:
 [[-10.03851316 -10.08802245 173.11583432]]


In [20]:
R.from_dcm(rot_in_room[0,...]).as_euler('ZYX', degrees=True)

In [25]:
r

array([ 173.04895325,   -7.05780819, -174.73349869])

In [ ]:
import time
while True:
    time.sleep(1)

[ 0 10 11 12]
[-41.82863998  48.68709946 418.32611084]
[-32.29256337  13.75156636 425.94291993]
[-32.36325634  13.4818728  188.94181891]

[-144.89752197 -804.35949707  317.53515625]
[-150.91511658 -747.40082779  430.11340978]
[-150.56714219 -647.74015399  193.17354491]

[-11702.38964844  -7855.84960938   4461.79833984]
[ 14363.42180907  -5606.73452978 -10492.91225307]
[ 14361.7692039   -5507.53729426 -10728.976911  ]

[-571.38989258  106.2677002   320.15924072]
[ 198.05850731 -367.67745711  516.49839972]
[ 199.24943827 -267.5206278   279.89568423]



In [421]:
T

array([  33.16356, -198.3498 ,  430.42578], dtype=float32)

In [422]:
to_marker_coord([0,0,0])

array([ 94.37149532, 173.23027303, 432.59809898])

In [423]:
to_room_coord([0,0,0])

array([ 94.37149532, 173.23027303, 197.59809898])